In [ ]:

import base64
import pandas as pd
import requests
import seaborn as sns
import spotipy

from dotenv import dotenv_values
from IPython.display import HTML, display
from matplotlib import pyplot
from time import sleep
from spotipy.oauth2 import SpotifyOAuth

In [ ]:
config = dotenv_values(".env")

In [ ]:
scope = "user-modify-playback-state app-remote-control streaming user-read-playback-state"

sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=config["SPOTIFY_CLIENT_ID"],
        client_secret=config["SPOTIFY_CLIENT_SECRET"],
        redirect_uri=config["SPOTIFY_REDIRECT_URL"],
        scope=scope
    )
)


In [ ]:
devices_df = pd.DataFrame(sp.devices()["devices"])
device_id = devices_df[devices_df["name"] == "Tim’s MacBook Pro (2)"]["id"].values[0]

In [ ]:
def display_two_images(img_url_1, img_url_2: str, text_1: str, text_2: str) -> None:
    """
    """
    html_template = """
        <div class="row">
            <div style="float:left;margin-right:30px;width:300px">{text_1}</div>
            <div style="float:left;margin-right:30px;width:300px">{text_2}</div>
        </div>
        <div style="clear:both"></div>
        <div class="row">
            <img style="float:left;margin-right:30px;" src="data:image/jpeg;base64,{b64_img_1}" width="300" height="300" />
            <img style="float:left;margin-right:30px;" src="data:image/jpeg;base64,{b64_img_2}" width="300" height="300" />
        </div>
    """
    img_bytes_1 = requests.get(img_url_1).content
    img_bytes_2 = requests.get(img_url_2).content
    b64_img_1 =  base64.b64encode(img_bytes_1).decode('ascii')
    b64_img_2 =  base64.b64encode(img_bytes_2).decode('ascii')
    txt = "sjfa;\nakd;f\nsdfa"
    display(
        HTML(
            html_template.format(
                text_1=text_1, text_2=text_2, b64_img_1=b64_img_1,b64_img_2=b64_img_2
            )
        )
    )


In [ ]:
def play_middle_of_track(device_id: str, track_series: pd.Series, n_seconds: int=9) -> None:
    """Play n_seconds on device starting from the middle of the track"""
    sp.start_playback(
        device_id=device_id,
        uris=[track_series["uri"]],
        position_ms=track_series["duration_ms"] / 2
    )
    sleep(n_seconds)
    sp.pause_playback(device_id=device_id)

def play_audio_feature_extremes(tracks_df, audio_feature, pause_seconds: int=2) -> None:
    """"""
    min_track = tracks_df.sort_values(by=audio_feature, ascending=True).iloc[0]
    max_track = tracks_df.sort_values(by=audio_feature, ascending=False).iloc[0]
    
    display_two_images(
        min_track["album_img_url"],
        max_track["album_img_url"],
        f"Lowest {audio_feature}({min_track[audio_feature]}): <br />{min_track['artist_names']} - {min_track['name']}",
        f"Highest {audio_feature}({max_track[audio_feature]}): <br />{max_track['artist_names']} - {max_track['name']}",
    )

    play_middle_of_track(device_id, min_track)

    sleep(pause_seconds)
    
    play_middle_of_track(device_id, max_track)

In [ ]:
df_apres_ski = pd.read_pickle("data/df_apres_ski.pkl")
df_top_2000 = pd.read_pickle("data/df_top_2000.pkl")

df_all_songs = (
    pd.concat(
        [
            df_apres_ski.assign(playlist="Après Ski"),
            df_top_2000.assign(playlist="Top 2000"),
        ]
    )
    .assign(duration_minutes=lambda track: track["duration_ms"] / 1000 / 60)
)

In [ ]:
df_apres_ski.head(1).T

In [ ]:
df_all_songs = (
    pd.concat(
        [
            df_apres_ski.assign(playlist="Après Ski"),
            df_top_2000.assign(playlist="Top 2000"),
        ]
    )
    .assign(duration_minutes=lambda track: track["duration_ms"] / 1000 / 60)
)

In [ ]:
zero_to_one_scale = [
    'acousticness',
    'danceability',
    'energy',
    'instrumentalness',
    'liveness',
    'speechiness',
    'valence',
]
other_features = ['duration_minutes', 'key', 'loudness', 'popularity', 'tempo']

In [ ]:
df_to_melt = df_all_songs[["id", "playlist"] + zero_to_one_scale + other_features]
df_to_melt.head()

In [ ]:
df_to_melt.shape

In [ ]:
melted_df = pd.melt(
    df_to_melt,
    id_vars=["id", "playlist"],
    value_vars=zero_to_one_scale + other_features,
    var_name="audio_feature",
    value_name="value",
)

In [ ]:
melted_df.head()

In [ ]:
melted_df.shape

In [ ]:
a4_dims = (11.7, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)
fig.suptitle('Zero-to-One Scale Audio Features')
sns.violinplot(ax=ax, data=melted_df[melted_df["audio_feature"].isin(zero_to_one_scale)], x="audio_feature", y="value", hue="playlist", split=True)
fig.savefig('images/zero_to_one_scale.png')

# What does this mean?!

In [ ]:
play_audio_feature_extremes(df_apres_ski, "acousticness")


In [ ]:
play_audio_feature_extremes(df_apres_ski, "danceability")

In [ ]:
# play_audio_feature_extremes(df_apres_ski, "energy")

In [ ]:
# play_audio_feature_extremes(df_apres_ski, "liveness")

In [ ]:
play_audio_feature_extremes(df_apres_ski, "instrumentalness")
# here we see that the lyrics don't need to be complex
# and sometimes we don't even need words at all

In [ ]:
# play_audio_feature_extremes(df_apres_ski, "speechiness")


In [ ]:
play_audio_feature_extremes(df_apres_ski, "valence")

In [ ]:
a4_dims = (11.7, 8.27)
fig, axs = pyplot.subplots(1, len(other_features), figsize=a4_dims)
fig.suptitle('Hetero-scale Audio Features')
for i, feature in enumerate(other_features):
    sns.violinplot(ax=axs[i], data=melted_df[melted_df["audio_feature"] == feature], x="audio_feature", y="value", hue="playlist", split=True)
fig.savefig('images/other_features.png')

In [ ]:
melted_df[melted_df["audio_feature"].isin(zero_to_one_scale)].groupby(["playlist", "audio_feature"]).mean(numeric_only=True)

In [ ]:
# a4_dims = (11.7, 8.27)
# fig, ax = pyplot.subplots(figsize=a4_dims)
# fig.suptitle('Zero-to-One Scale Audio Features')
g = sns.catplot(
    data=melted_df[melted_df["audio_feature"].isin(zero_to_one_scale)],
    kind="bar",
    x="audio_feature", y="value", hue="playlist",
    # height=6,
    # width=11.7,
)
# Draw a nested barplot by species and sex
# g = sns.catplot(
#     data=melted, kind="bar",
#     x="species", y="body_mass_g", hue="sex",
#     errorbar="sd", palette="dark", alpha=.6, height=6
# )
# g.despine(left=True)
# g.set_axis_labels("", "Body mass (g)")
# g.legend.set_title("")


# fig.savefig('other_features.png')


In [ ]:
# Niet-woorden: in Hurra die Gams en in Atemlos durch die Nach

In [ ]:
(
    df_apres_ski.query("popularity >= 40")
    .sort_values("danceability", ascending=False)
)

In [ ]:
(
    df_apres_ski.query("popularity >= 40")
    .sort_values("acousticness", ascending=False)
)